<a href="https://colab.research.google.com/github/JotaBlanco/QuixStreamsNotebooks/blob/main/Conferences/BerlinTimeseriesMeetup/Quix_Streams_SUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Quix Streams
Just use pip install to download the Quix Streams library. 

[Quix Streams](https://github.com/quixio/quix-streams) is an open source Python library for processing streaming data. It’s aimed at people who work with time-series data streams — from developers and ML engineers to data scientists and data engineers.

In [ ]:
! pip install quixstreams

# Import the libraries
We will be using mainly pandas, quix, matplotlib and seaborn.

In [ ]:
import pandas as pd
import quixstreams as qx

# 1 - Create client
Let's start by creating a Quix client that we'll use to publish and subscribe to Kafka topics.

In [ ]:
token = 'sdk-296f2b9decff4770a525ff7d8855a78d'
client = qx.QuixStreamingClient(token)
client

# 2 - Consumer client
To suscribe to data from one topic, we will need to create a consumer client pointing to that topic.

In [ ]:
topic_name = "test-topic"
topic_consumer = client.get_topic_consumer(topic_name)
topic_consumer

# 3 - Suscribing to topics
Once you have the TopicConsumer instance you can start receiving data. These are the steps:

## 3.1 - Subscribing to streams
For each stream received, the TopicConsumer will execute the callback you define. This callback will be invoked every time you receive a new stream.

In [ ]:
def on_stream_received_handler(stream_received: qx.StreamConsumer):
  """
  My callback to new streams received is defined here
  """
  print("New stream just received: " + stream_received.stream_id)

topic_consumer = client.get_topic_consumer(topic_name)
topic_consumer.on_stream_received = on_stream_received_handler
qx.App.run()

## 3.2 - Subscribing to Timeseries data
You can subscribe to time-series data from streams using the on_data_received callback of the StreamConsumer instance.

### 3.2.1 - qx.TimeseriesData
This is how you read the data in the standard TimeseriesData format:

In [ ]:
def on_stream_received_handler(stream_received: qx.StreamConsumer):
    stream_received.timeseries.on_data_received = on_timeseries_data_received_handler

def on_timeseries_data_received_handler(stream: qx.StreamConsumer, data: qx.TimeseriesData):
  print("Data from stream " + stream.stream_id)
  with data:
    print(data)

topic_consumer = client.get_topic_consumer(topic_name)
topic_consumer.on_stream_received = on_stream_received_handler
qx.App.run()

### 3.2.2 - pd.DataFrame
This is how you read the data in pandas dataframe format.

In [ ]:
def on_stream_received_handler(stream_received: qx.StreamConsumer):
  stream_received.timeseries.on_dataframe_received = on_timeseries_data_received_handler

def on_timeseries_data_received_handler(stream: qx.StreamConsumer, df: pd.DataFrame):
  print("Data from stream " + stream.stream_id)
  display(df)

topic_consumer = client.get_topic_consumer(topic_name)
topic_consumer.on_stream_received = on_stream_received_handler
qx.App.run()

In [ ]:
df= pd.DataFrame()

def on_stream_received_handler(stream_received: qx.StreamConsumer):
  stream_received.timeseries.on_dataframe_received = on_timeseries_data_received_handler

def on_timeseries_data_received_handler(stream: qx.StreamConsumer, df_i: pd.DataFrame):
  global df
  df = df.append(df_i)
  print("Data from stream " + stream.stream_id)
  display(df_i)

topic_consumer = client.get_topic_consumer(topic_name)
topic_consumer.on_stream_received = on_stream_received_handler
qx.App.run()

In [ ]:
df